In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
import io
import requests
import math
from scipy import stats

In [2]:
def feature_normalize(dataset):
    mu = np.mean(dataset,axis=0)
    sigma = np.std(dataset,axis=0)
    
    print(df.head(3))
    return (dataset - mu)/sigma

In [3]:
def str_to_int(df):
    str_columns = df.select_dtypes(['object']).columns
    print(str_columns)
    for col in str_columns:
        df[col] = df[col].astype('category')

    cat_columns = df.select_dtypes(['category']).columns
    df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
    return df

In [4]:
def count_space_except_nan(x):
    if isinstance(x,str):
        return x.count(" ") + 1
    else :
        return 0

In [5]:
def one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        del df[each]
        df = pd.concat([df, dummies], axis=1)
        
    print(df.head())
    return df

In [15]:
df_train = pd.read_csv('./titanic/train.csv')

In [16]:
df_train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [17]:
df_test = pd.read_csv('./titanic/test.csv')

In [18]:
df_test.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [19]:
print (df_train.isnull().sum())
delete_columns = ["Ticket", "Name", "PassengerId", "Cabin", "Embarked"]

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [20]:
def pre_processing(df):
    df.head(1)
    df.drop(delete_columns, axis=1, inplace=True)
    # Count room nubmer
    # df_train["Cabin"] = df_train["Cabin"].apply(count_space_except_nan)
    # Replace NaN with mean value
    df["Age"].fillna(df["Age"].mean(), inplace=True)
    # Pclass, Embarked one-hot
    df = one_hot(df, df.loc[:, ["Pclass"]].columns)
    # String to int
    df = str_to_int(df)
    # Age Normalization
    #df["Age"] = feature_normalize(df["Age"])
    stats.describe(df).variance
    return df

In [21]:
df_train = pre_processing(df_train)


   Survived     Sex   Age  SibSp  Parch     Fare  Pclass_1  Pclass_2  Pclass_3
0         0    male  22.0      1      0   7.2500         0         0         1
1         1  female  38.0      1      0  71.2833         1         0         0
2         1  female  26.0      0      0   7.9250         0         0         1
3         1  female  35.0      1      0  53.1000         1         0         0
4         0    male  35.0      0      0   8.0500         0         0         1
Index(['Sex'], dtype='object')


In [22]:
#save PassengerId for evaluation
test_passenger_id = df_test["PassengerId"]
df_test = pre_processing(df_test)

      Sex   Age  SibSp  Parch     Fare  Pclass_1  Pclass_2  Pclass_3
0    male  34.5      0      0   7.8292         0         0         1
1  female  47.0      1      0   7.0000         0         0         1
2    male  62.0      0      0   9.6875         0         1         0
3    male  27.0      0      0   8.6625         0         0         1
4  female  22.0      1      1  12.2875         0         0         1
Index(['Sex'], dtype='object')


In [23]:
features = df_train.iloc[:, 1:].values
# features = feature_normalize(features)
labels = df_train.iloc[:, :1].values
print(features.shape, labels.shape)
stats.describe(features).variance

(891, 8) (891, 1)


array([2.28474508e-01, 1.69052400e+02, 1.21604308e+00, 6.49728244e-01,
       2.46943685e+03, 1.83861083e-01, 1.64047466e-01, 2.47670210e-01])

In [24]:
stats.describe(features).mean

array([ 0.64758698, 29.69911765,  0.52300786,  0.38159371, 32.20420797,
        0.24242424,  0.20650954,  0.55106622])

In [25]:
real_test_x = df_test.values  # Why numpy array??
print(real_test_x.shape)

(418, 8)


In [26]:
type(real_test_x)

numpy.ndarray

In [27]:
real_test_x.shape

(418, 8)

In [28]:
len(features)

891

In [29]:
rnd_indices = np.random.rand(len(features)) < 0.80  #Why??
print(rnd_indices)

[ True  True  True False  True  True  True  True False  True  True  True
  True False  True  True  True  True  True False  True False  True  True
  True  True  True False False  True False  True  True False False  True
  True False  True  True  True  True  True  True False False  True False
 False  True False  True  True  True  True  True False  True False  True
  True  True False  True  True  True  True  True  True  True  True  True
 False  True  True False  True  True  True  True  True False False False
  True  True  True  True  True  True False  True  True  True  True False
 False  True False False False  True False  True False  True  True False
  True  True  True  True False  True  True  True  True  True  True  True
  True  True False  True  True  True  True  True  True  True  True False
 False  True  True  True  True False  True False  True False False  True
  True  True  True  True  True False False False False  True  True  True
  True  True  True  True  True  True  True  True  T

In [30]:
real_test_x[0]

array([ 1.    , 34.5   ,  0.    ,  0.    ,  7.8292,  0.    ,  0.    ,
        1.    ])

In [31]:
train_x = features[rnd_indices]
train_y = labels[rnd_indices]
test_x = features[~rnd_indices]
test_y = labels[~rnd_indices]

feature_count = train_x.shape[1]
label_count = train_y.shape[1]
print(feature_count, label_count)

8 1


In [32]:
# inputs
training_epochs = 3000
learning_rate = 0.01
n_hidden_1 = 50 
n_hidden_2 = 50
n_hidden_3 = 25
hidden_layers = feature_count - 1
num_classes = 1
cost_history = np.empty(shape=[1],dtype=float)

In [33]:
# Placeholders
X = tf.placeholder(tf.float32,[None,feature_count])
Y = tf.placeholder(tf.float32,[None,label_count])

In [34]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([feature_count, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [35]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 50 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 50 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Hidden fully connected layer with 25 neurons
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return out_layer

In [36]:
# Construct model
logits = neural_net(X)
prediction = tf.nn.sigmoid(logits)

In [37]:
# loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
#     logits=logits, labels=Y))
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [38]:
correct_pred = tf.equal(tf.round(prediction), Y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [39]:
def divide_batches(input_batch, batch_size):
    """
    Divide into batches

    :param input_batch:
    :param batch_size:
    :return:
    """
    output_batch = []
    for i in range(0, len(input_batch), batch_size):
        output_batch.append(input_batch[i: i + batch_size])
    return output_batch

In [40]:
train_batches = divide_batches(train_x,64)
train_labels_batches = divide_batches(train_y,64)

In [41]:
print(len(train_labels_batches[0][0]))

1


In [42]:
display_step = 1000

In [61]:
def sigmoid(z):
    return 1 / (1 + math.exp(-z))


# def calculate_confusion_matrix(y_true, y_pred, threshold):
#     #y_pred = sigmoid(y_pred)
#     y_pred[np.where(y_pred >= threshold)] = 1.0
#     y_pred[np.where(y_pred < threshold)] = 0.0
#     return confusion_matrix(y_true, y_pred)

from sklearn.metrics import confusion_matrix
threshold = 0.5

In [62]:
init = tf.global_variables_initializer()

In [67]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_epochs+1):
        train_predictions = []
        for i,_ in enumerate(train_batches) :
            #batch_x, batch_y = mnist.train.next_batch(batch_size)
            batch_x = train_batches[i]
            #reshape_batch_x = tf.reshape(batch_x,shape=[-1, len(batch_x[0])])
            batch_y = train_labels_batches[i]            
            # Run optimization op (backprop)
            sess.run(optimizer, feed_dict={X: batch_x, Y: batch_y})
            
            model_prediction = sess.run(prediction,feed_dict={X: batch_x,Y: batch_y})
            train_predictions.append(temp for temp in model_prediction)
            
            if step % display_step == 0 or step == 1:
                # Calculate batch loss and accuracy
                cp,loss, acc = sess.run([correct_pred,cost, accuracy], feed_dict={X: batch_x,
                                                                     Y: batch_y})
                print("Step " + str(step) + ", Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))
  
                #print("Model Predictions : ",model_prediction)
         
        train_predictions = [item for sublist in train_predictions for item in sublist]
        train_predictions = np.asarray(train_predictions).flatten()
        trans_train_label = np.asarray([item for sublist in train_y for item in sublist]).flatten()

        print('Len trans_train_label',len(trans_train_label))
        print('Len train_predictions',len(train_predictions))
        
        
        
        train_predictions[np.where(train_predictions >= threshold)] = 1.0
        train_predictions[np.where(train_predictions < threshold)] = 0.0
        train_conf_matrix_default = confusion_matrix(trans_train_label, train_predictions)
        

        print('Train Confusion Matrix - 0.5 *********')
        print(train_conf_matrix_default)
#         print(test_conf_matrix_default)

    p,acc = sess.run([prediction,accuracy], feed_dict={X: test_x,
                                      Y: test_y})

    test_predictions = np.asarray(p).flatten()
    trans_test_label = np.asarray([item for sublist in test_y for item in sublist]).flatten()

    test_predictions[np.where(test_predictions >= threshold)] = 1.0
    test_predictions[np.where(test_predictions < threshold)] = 0.0
    test_conf_matrix_default = confusion_matrix(trans_test_label, test_predictions)

    print("Test Accuracy:  ",acc)
    print("Test Confusion matrix:  ",test_conf_matrix_default)
 
    

Step 1, Minibatch Loss= 2171.3386, Training Accuracy= 0.422
Step 1, Minibatch Loss= 2030.4731, Training Accuracy= 0.234
Step 1, Minibatch Loss= 434.6417, Training Accuracy= 0.719
Step 1, Minibatch Loss= 2306.2754, Training Accuracy= 0.500
Step 1, Minibatch Loss= 1525.7205, Training Accuracy= 0.594
Step 1, Minibatch Loss= 1133.6611, Training Accuracy= 0.625
Step 1, Minibatch Loss= 898.9585, Training Accuracy= 0.688
Step 1, Minibatch Loss= 561.6001, Training Accuracy= 0.797
Step 1, Minibatch Loss= 1098.1892, Training Accuracy= 0.625
Step 1, Minibatch Loss= 1058.0974, Training Accuracy= 0.594
Step 1, Minibatch Loss= 597.4604, Training Accuracy= 0.571
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[261 169]
 [124 142]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[318 112]
 [146 120]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[294 136]
 [137 129]]
Le

Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[281 149]
 [ 84 182]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[330 100]
 [ 98 168]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[340  90]
 [112 154]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[353  77]
 [104 162]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[291 139]
 [ 92 174]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[365  65]
 [147 119]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[297 133]
 [ 85 181]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[358  72]
 [123 143]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *******

Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[343  87]
 [ 68 198]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[347  83]
 [ 98 168]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[323 107]
 [103 163]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[309 121]
 [ 79 187]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[320 110]
 [116 150]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[359  71]
 [ 97 169]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[323 107]
 [ 98 168]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[359  71]
 [137 129]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *******

Train Confusion Matrix - 0.5 *********
[[346  84]
 [ 99 167]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[353  77]
 [ 87 179]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[315 115]
 [ 93 173]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[302 128]
 [ 99 167]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[335  95]
 [110 156]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[349  81]
 [ 82 184]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[341  89]
 [ 78 188]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[313 117]
 [132 134]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[333  97]
 [ 78 188]]
Len trans_train_label 696


Train Confusion Matrix - 0.5 *********
[[323 107]
 [118 148]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[341  89]
 [ 86 180]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[324 106]
 [ 98 168]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[327 103]
 [ 88 178]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[309 121]
 [ 94 172]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[370  60]
 [108 158]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[319 111]
 [ 82 184]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[331  99]
 [ 99 167]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[345  85]
 [ 87 179]]
Len trans_train_label 696


...

Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 80 186]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 80 186]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 80 186]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 80 186]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 80 186]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 80 186]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 80 186]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 80 186]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *******

Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[324 106]
 [ 89 177]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[365  65]
 [ 99 167]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [103 163]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[345  85]
 [ 98 168]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[329 101]
 [ 75 191]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[359  71]
 [104 162]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[360  70]
 [ 71 195]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[372  58]
 [ 88 178]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *******

Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[366  64]
 [ 77 189]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[366  64]
 [ 77 189]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[366  64]
 [ 77 189]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[366  64]
 [ 77 189]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[366  64]
 [ 77 189]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[366  64]
 [ 77 189]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[366  64]
 [ 77 189]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[366  64]
 [ 77 189]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *******

Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 79 187]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 79 187]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 79 187]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 79 187]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 79 187]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 79 187]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 79 187]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *********
[[368  62]
 [ 79 187]]
Len trans_train_label 696
Len train_predictions 696
Train Confusion Matrix - 0.5 *******

Test Accuracy:   0.774359
Test Confusion matrix:   [[97 22]
 [22 54]]
